In [7]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os

In [8]:
# Cargar variables desde .env
load_dotenv()

# Obtener los valores
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
HOST = os.getenv("DB_HOST")
PORT = os.getenv("DB_PORT")
DATABASE = os.getenv("DB_NAME")

In [9]:
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

In [10]:
df = pd.read_sql("SELECT NOW();", engine)
print(df)

                               now
0 2025-03-30 15:21:11.501463+00:00
